In [1]:
import scanpy as sc
import plotly.express as px
import numpy as np
import pandas as pd

from tqdm import tqdm 
from itertools import product

from benchdb import * 
from benchmodels import *
from benchplots import *
from benchutils import *

%load_ext autoreload
%autoreload 2

/Users/valerio/Desktop/refcm_github/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Seed set to 0


In [ ]:
db = load_benchdb("bench.json")
# tmf = load_benchdb("tmuris_facs.json")
tmd = load_benchdb("tmuris_drop.json")


mvote_all(db, to_mvote=RM_METHODS)
# mvote_all(tmf, to_mvote=RM_METHODS)
mvote_all(tmd, to_mvote=RM_METHODS)

df = pd.concat([bench_to_df(db), bench_to_df(tmd)])

## pairwise matrices

In [4]:
fig = plot_pairwise(df, 'SVM', "PBMC Bench1", zmin=0, text_auto=False, title="SVM")
add_paper_styling(fig, lines=False)

In [5]:
panel = plot_pairwise_panel(
    df, RCM_METHODS, 'PBMC Bench1', 
    rows=1, cols=4, 
    xtitle_y_off=-0.4, ytitle_x_off=-0.10
)
panel.update_layout(height=500, width = 1400)
panel

## aggregated box plots

In [6]:
fig = plot_box(df, 'Tabula Muris Senis', merge_mv_rcm=False, horizontal=True)
fig

In [7]:
fig = plot_box_panel(df, ['scIB pancreas', 'PBMC Bench1'], horizontal=True, color_boxes=True)
fig.update_layout(height=500, width=1000)


## heatmap

In [4]:
# Single heatmap
fig = plot_heatmap(df, dataset="Allen Brain")
fig.show()

In [6]:
df

,method,mtype,dataset,q,ref,task,acc
0,SVM,RM,scIB pancreas,celseq2,celseq,celseq2 | celseq,0.965427
1,SVM,RM,scIB pancreas,celseq2,fluidigmc1,celseq2 | fluidigmc1,0.972867
2,SVM,RM,scIB pancreas,celseq2,inDrop1,celseq2 | inDrop1,0.898468
3,SVM,RM,scIB pancreas,celseq2,inDrop2,celseq2 | inDrop2,0.860832
4,SVM,RM,scIB pancreas,celseq2,inDrop3,celseq2 | inDrop3,0.934792
...,...,...,...,...,...,...,...
95,MV-scmapcluster,MV,Tabula Muris Senis,fold 1,fref,fold 1,0.753306
96,MV-scmapcluster,MV,Tabula Muris Senis,fold 2,fref,fold 2,0.751384
97,MV-scmapcluster,MV,Tabula Muris Senis,fold 3,fref,fold 3,0.851139
98,MV-scmapcluster,MV,Tabula Muris Senis,fold 4,fref,fold 4,0.881650


In [5]:
fig = plot_heatmap_panel(
    df,
    datasets=["Allen Brain", "Tabula Muris Senis", "Embryo", "Adrenal", "CellBench"],
    subtitle_fs=14
)
fig.show()


## with marginals

In [69]:
fig = plot_heatmap_with_marginals(df, dataset="PBMC Bench1")
fig.show()

## Figure plots

In [5]:
morder = df[['method', 'acc']].groupby("method").mean().sort_values('acc', ascending=False).index.tolist()

In [6]:
fig = plot_heatmap_panel(
    df,
    datasets=["Allen Brain", "Tabula Muris Senis", "Embryo", "Adrenal", "CellBench"],
    methods=morder,
    reorder_by_mean=False,
    height=600,
    width=1000
)

for ann in fig.layout.annotations:
    ann.yshift = 4

fig.show()
fig.write_image('bench.png', scale=3)

In [6]:
fig = plot_box_panel(
    df, 
    datasets=['scIB pancreas', 'PBMC Bench1'], 
    methods=morder,
    reorder_by_median=False, 
    horizontal=True, 
    color_boxes=False
)
fig.update_layout( 
    height=500, 
    width=1000
)


for ann in fig.layout.annotations:
    ann.yshift = 4

# fig.update_yaxes(title="")
fig.show()
fig.write_image('boxes.png', scale=3)